# Introduction

This notebook predicts the `beer_style` using a neural network on the PyTorch
framework. This notebook uses a more complex version of `2_pytorch`.

## Summary
The [classification report](#Classification-report) shows that
* the test accuracy is 30%, while the validation accuracy is also around 30%,
hence there doesn't seem to be overfitting
* the [top 10 classes by number of observations](#Top-10-by-number-of-observations)
 don't have high f1-scores.
* the [top 10 classes by f1-score](#Top-10-by-f1-score) have very few
observations

In [1]:
artefact_prefix = '3_pytorch'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from datetime import datetime
import pandas as pd
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from category_encoders.binary import BinaryEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from joblib import dump, load

from src.data.sets import save_sets
from src.data.sets import load_sets
from src.data.sets import split_sets_random
from src.data.sets import test_class_exclusion
from src.models.performance import convert_cr_to_dataframe
from src.models.pytorch import PytorchClassification_3
from src.models.pytorch import get_device
from src.models.pytorch import train_classification
from src.models.pytorch import test_classification
from src.models.pytorch import PytorchDataset
from src.models.pipes import create_preprocessing_pipe
from src.visualization.visualize import plot_confusion_matrix

# Set up directories

In [4]:
project_dir = Path.cwd().parent
data_dir = project_dir / 'data'
raw_data_dir = data_dir / 'raw'
interim_data_dir = data_dir / 'interim'
processed_data_dir = data_dir / 'processed'
reports_dir = project_dir / 'reports'
models_dir = project_dir / 'models'

# Load data

In [5]:
X_train, X_test, X_val, y_train, y_test, y_val = load_sets()

In [6]:
y_train.value_counts(normalize=True)

American IPA                        0.074216
American Double / Imperial IPA      0.054196
American Pale Ale (APA)             0.040204
Russian Imperial Stout              0.034057
American Double / Imperial Stout    0.031940
                                      ...   
Gose                                0.000444
Faro                                0.000378
Roggenbier                          0.000297
Kvass                               0.000189
Happoshu                            0.000136
Name: beer_style, Length: 104, dtype: float64

In [7]:
y_val.value_counts(normalize=True)

American IPA                        0.074306
American Double / Imperial IPA      0.054282
American Pale Ale (APA)             0.040076
Russian Imperial Stout              0.034400
American Double / Imperial Stout    0.031867
                                      ...   
English Pale Mild Ale               0.000432
Faro                                0.000416
Roggenbier                          0.000258
Happoshu                            0.000208
Kvass                               0.000170
Name: beer_style, Length: 104, dtype: float64

In [8]:
y_test.value_counts(normalize=True)

American IPA                        0.073603
American Double / Imperial IPA      0.054074
American Pale Ale (APA)             0.039326
Russian Imperial Stout              0.034010
American Double / Imperial Stout    0.032103
                                      ...   
Gose                                0.000375
Faro                                0.000369
Roggenbier                          0.000318
Kvass                               0.000199
Happoshu                            0.000145
Name: beer_style, Length: 104, dtype: float64

In [9]:
X_train

,brewery_name,review_aroma,review_appearance,review_palate,review_taste
0,"Kirin Brewery Company, Limited",1.5,3.0,3.0,3.5
1,Huisbrouwerij Klein Duimpje,3.0,4.0,3.5,3.5
2,Southampton Publick House,3.0,3.5,4.0,3.5
3,Rock Bottom Restaurant & Brewery,3.5,4.0,2.5,3.5
4,Boston Beer Company (Samuel Adams),4.0,3.5,3.5,3.5
...,...,...,...,...,...
951963,Bierbrouwerij Sint Christoffel B.V,4.5,4.0,4.0,4.0
951964,Brouwerij Slaghmuylder,3.5,4.0,3.5,4.0
951965,Thirsty Dog Brewing Company,4.0,4.0,4.0,4.5
951966,OPA-OPA Steakhouse & Brewery,4.0,3.5,3.0,4.0


Check for excluded classes.

In [10]:
test_class_exclusion(y_train, y_test, y_val)

'✔ All the sets contain all the classes.'

# Preprocess data

1. The `brewery_name` is a feature with a very high cardinality, ~5700. One hot encoding is not feasible as it will introduce 5700 very sparse columns. Another option is to use binary encoding, which would result in 14 new columns.
1. Standard scaling is used to ensure that the binary columns ([0, 1])and the review columns ([1, 5]) are on the same scale.

In [11]:
pipe = Pipeline([
    ('bin_encoder', BinaryEncoder(cols=['brewery_name'])),
    ('scaler', StandardScaler())
])

In [12]:
X_train_trans = pipe.fit_transform(X_train)
X_val_trans = pipe.transform(X_val)
X_test_trans = pipe.transform(X_test)

In [13]:
X_train_trans.shape

(951968, 18)

In [14]:
n_features = X_train_trans.shape[1]
n_features

18

In [15]:
n_classes = y_train.nunique()
n_classes

104

## Encoding

PyTorch accepts only numerical labels.

In [16]:
le = LabelEncoder()
y_train_trans = le.fit_transform(y_train.to_frame())
y_val_trans = le.fit_transform(y_val.to_frame())
y_test_trans = le.transform(y_test.to_frame())

C:\Users\Roger\.conda\envs\adsi_ass_2\lib\site-packages\sklearn\preprocessing\_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Roger\.conda\envs\adsi_ass_2\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [17]:
y_test_trans

array([98, 89,  2, ..., 37, 94, 98])

## Convert to Pytorch tensors

In [18]:
device = get_device()
device

device(type='cuda', index=0)

In [19]:
train_dataset = PytorchDataset(X=X_train_trans, y=y_train_trans)
val_dataset = PytorchDataset(X=X_val_trans, y=y_val_trans)
test_dataset = PytorchDataset(X=X_test_trans, y=y_test_trans)

# Classification model

In [20]:
model = PytorchClassification_3(n_features=n_features, n_classes=n_classes)

In [21]:
model.to(device)

PytorchClassification_3(
  (layer_1): Linear(in_features=18, out_features=512, bias=True)
  (batchnorm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer_2): Linear(in_features=512, out_features=128, bias=True)
  (batchnorm2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer_3): Linear(in_features=128, out_features=64, bias=True)
  (batchnorm3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer_out): Linear(in_features=64, out_features=104, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
)

In [22]:
criterion = nn.CrossEntropyLoss()

In [223]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model

In [224]:
N_EPOCHS = 20
BATCH_SIZE = 512
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

In [225]:
start_time = datetime.now()
print(f'Started: {start_time}')
for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_classification(train_dataset,
                                                 model=model,
                                                 criterion=criterion, 
                                                 optimizer=optimizer,
                                                 batch_size=BATCH_SIZE,
                                                 device=device,
                                                 scheduler=scheduler)
    valid_loss, valid_acc = test_classification(val_dataset,
                                                model=model,
                                                criterion=criterion, 
                                                batch_size=BATCH_SIZE, 
                                                device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\tLoss: {train_loss:.4f}\t|\tAcc: {train_acc * 100:.1f}%')
    print(f'\t(valid)\tLoss: {valid_loss:.4f}\t|\tAcc: {valid_acc * 100:.1f}%')

end_time = datetime.now()
runtime = end_time - start_time
print(f'Ended: {end_time}')
print(f'Runtime: {runtime}')


Started: 2021-03-12 16:20:13.922325
Epoch: 0
	(train)	Loss: 0.0064	|	Acc: 19.5%
	(valid)	Loss: 0.0055	|	Acc: 25.9%
Epoch: 1
	(train)	Loss: 0.0057	|	Acc: 23.7%
	(valid)	Loss: 0.0053	|	Acc: 27.0%
Epoch: 2
	(train)	Loss: 0.0056	|	Acc: 24.6%
	(valid)	Loss: 0.0051	|	Acc: 27.8%
Epoch: 3
	(train)	Loss: 0.0055	|	Acc: 25.1%
	(valid)	Loss: 0.0051	|	Acc: 28.2%
Epoch: 4
	(train)	Loss: 0.0054	|	Acc: 25.4%
	(valid)	Loss: 0.0050	|	Acc: 28.2%
Epoch: 5
	(train)	Loss: 0.0054	|	Acc: 25.7%
	(valid)	Loss: 0.0050	|	Acc: 28.5%
Epoch: 6
	(train)	Loss: 0.0053	|	Acc: 25.9%
	(valid)	Loss: 0.0050	|	Acc: 28.6%
Epoch: 7
	(train)	Loss: 0.0053	|	Acc: 26.0%
	(valid)	Loss: 0.0049	|	Acc: 28.8%
Epoch: 8
	(train)	Loss: 0.0053	|	Acc: 26.1%
	(valid)	Loss: 0.0049	|	Acc: 29.0%
Epoch: 9
	(train)	Loss: 0.0053	|	Acc: 26.2%
	(valid)	Loss: 0.0049	|	Acc: 29.0%
Epoch: 10
	(train)	Loss: 0.0052	|	Acc: 26.4%
	(valid)	Loss: 0.0049	|	Acc: 29.1%
Epoch: 11
	(train)	Loss: 0.0052	|	Acc: 26.5%
	(valid)	Loss: 0.0049	|	Acc: 29.2%
Epoch: 12
	(tr

# Prediction

In [226]:
preds = model(test_dataset.X_tensor.to(device)).argmax(1)
preds

tensor([25, 18,  9,  ..., 65, 47, 25], device='cuda:0')

# Evaluation

## Classification report

In [227]:
report = classification_report(y_test, le.inverse_transform(preds.cpu()))
print(report)

C:\Users\Roger\.conda\envs\adsi_ass_2\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                     precision    recall  f1-score   support

                            Altbier       0.35      0.34      0.35      1521
             American Adjunct Lager       0.53      0.73      0.62      6085
           American Amber / Red Ale       0.19      0.22      0.20      9288
         American Amber / Red Lager       0.31      0.32      0.31      1887
                American Barleywine       0.24      0.01      0.03      5390
                 American Black Ale       0.40      0.05      0.08      2394
                American Blonde Ale       0.22      0.01      0.02      2594
                 American Brown Ale       0.26      0.08      0.12      5066
            American Dark Wheat Ale       0.00      0.00      0.00       296
     American Double / Imperial IPA       0.25      0.36      0.30     17159
 American Double / Imperial Pilsner       0.00      0.00      0.00      1109
   American Double / Imperial Stout       0.34      0.46      0.39     1018

In [141]:
report_dict = classification_report(y_test,
                                    le.inverse_transform(preds.cpu()),
                                    output_dict=True)
report_df = convert_cr_to_dataframe(report_dict)

C:\Users\Roger\.conda\envs\adsi_ass_2\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Top 10 by number of observations

In [233]:
report_df.sort_values(by=['support'], ascending=False).head(10)

,beer_style,precision,recall,f1,support
12,American IPA,0.180058,0.511835,0.266400,23406
9,American Double / Imperial IPA,0.255797,0.365666,0.301020,17196
14,American Pale Ale (APA),0.167338,0.180754,0.173787,12647
89,Russian Imperial Stout,0.309647,0.404610,0.350816,10630
17,American Porter,0.223503,0.207952,0.215447,10161
11,American Double / Imperial Stout,0.361799,0.448238,0.400407,10104
2,American Amber / Red Ale,0.171532,0.228809,0.196073,9143
25,Belgian Strong Dark Ale,0.386467,0.396476,0.391407,7491
60,Fruit / Vegetable Beer,0.259689,0.334737,0.292476,6886
19,American Strong Ale,0.241337,0.365169,0.290611,6408


## Top 10 by f1-score

In [237]:
report_df.sort_values(by=['f1'], ascending=False).head(10)

,beer_style,precision,recall,f1,support
96,Scottish Gruit / Ancient Herbed Ale,0.863946,0.718868,0.784758,530
69,Japanese Rice Lager,0.665782,0.789308,0.722302,318
58,Flanders Red Ale,0.840642,0.593208,0.695575,1325
74,Lambic - Fruit,0.539014,0.726476,0.618861,2168
87,Rauchbier,0.707993,0.547289,0.617354,793
1,American Adjunct Lager,0.499625,0.761640,0.603417,6121
53,Euro Pale Lager,0.519273,0.696912,0.595120,3692
62,Gose,0.591304,0.523077,0.555102,130
67,Irish Dry Stout,0.503618,0.577736,0.538137,2650
76,Light Lager,0.489424,0.466458,0.477665,2877


## Top 10 by precision

In [238]:
report_df.sort_values(by=['precision'], ascending=False).head(10)

,beer_style,precision,recall,f1,support
30,Black & Tan,0.985714,0.146809,0.255556,470
13,American Malt Liquor,0.928994,0.183626,0.306641,855
34,Chile Beer,0.926316,0.176000,0.295798,500
96,Scottish Gruit / Ancient Herbed Ale,0.863946,0.718868,0.784758,530
58,Flanders Red Ale,0.840642,0.593208,0.695575,1325
91,Sahti,0.756098,0.136564,0.231343,227
29,Bière de Garde,0.716194,0.312682,0.435312,1372
87,Rauchbier,0.707993,0.547289,0.617354,793
36,Czech Pilsener,0.685613,0.304536,0.421743,2535
77,Low Alcohol Beer,0.678571,0.075397,0.135714,252


## Top 10 by recall

In [239]:
report_df.sort_values(by=['recall'], ascending=False).head(10)

,beer_style,precision,recall,f1,support
69,Japanese Rice Lager,0.665782,0.789308,0.722302,318
1,American Adjunct Lager,0.499625,0.761640,0.603417,6121
74,Lambic - Fruit,0.539014,0.726476,0.618861,2168
96,Scottish Gruit / Ancient Herbed Ale,0.863946,0.718868,0.784758,530
53,Euro Pale Lager,0.519273,0.696912,0.595120,3692
58,Flanders Red Ale,0.840642,0.593208,0.695575,1325
67,Irish Dry Stout,0.503618,0.577736,0.538137,2650
87,Rauchbier,0.707993,0.547289,0.617354,793
62,Gose,0.591304,0.523077,0.555102,130
12,American IPA,0.180058,0.511835,0.266400,23406


# Save objects for production

## Save model

In [234]:
path = models_dir / f'{artefact_prefix}_model'
torch.save(model, path.with_suffix('.torch'))

## Create pipe object

This is for transforming the input prior to prediction.

In [235]:
X = pd.concat([X_train, X_val, X_test])
prod_pipe = create_preprocessing_pipe(X)

path = models_dir / f'{artefact_prefix}_pipe'
dump(prod_pipe, path.with_suffix('.sav'))

['D:\\git\\assignment_2\\models\\2_pytorch_pipe.sav']

## Save `LabelEncoder`

This is required to get back the name of the name of the `beer_style`.

In [236]:
path = models_dir / f'{artefact_prefix}_label_encoder'
dump(le, path.with_suffix('.sav'))


['D:\\git\\assignment_2\\models\\2_pytorch_label_encoder.sav']